# Troubleshooting, finding the proper path

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
chrome_path = 'C:\\Users\\nwang\\chromedriver.exe'
service = Service(chrome_path)
driver = webdriver.Chrome(service=service)

#testing on fresh produce fruit
url = "https://www.target.com/s?searchTerm=fresh+produce+fruit&category=0%7CAll%7Cmatchallpartial%7Call+categories&tref=typeahead%7Cterm%7C1%7Cfresh+produce+fruit%7Cfresh+produce+fruit%7C%7C%7Cservice%7C%7C%7C%7C%7Cfacets_sb&searchTermRaw=fresh+produce"
driver.get(url)
time.sleep(8)  #wait for page to load

#scroll
for _ in range(5):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)


#find product elements
products = driver.find_elements(By.XPATH, '//a[@data-test="product-title"]')
#price = product.find_element(By.XPATH, "//span[@data-test='current-price']").text
print(f"Found {len(products)} products.")  #debugging output
price = driver.find_elements(By.XPATH, ".//span[@data-test='current-price']//span")
print(f"Found {len(price)} prices.")
for product in products:
    name = product.get_attribute("aria-label")
    print(name)  #print product name

for price in price:
    name = price.text
    print(name)

#clean up and close the driver
driver.quit()


Found 24 products.
Found 24 prices.
Strawberries - 1lb
Extra Large Green Seedless Grapes - 1.5lb Bag
Sun Beams Mandarin Oranges - 3lb - Good & Gather™
Banana - each - Good & Gather™
Extra Large Red Seedless Grapes - 1.5lb Bag
Blueberries - 11.2oz
Organic Bananas - 2lb - Good & Gather™
Navel Orange - each
Lemons - 2lb Bag - Good & Gather™
Blackberries - 6oz
Avocado - each
Organic Gala Apples - 2lb Bag - Good & Gather™
Lemon - each
Lime - each
Tri-Colored Bell Peppers - 16oz/3ct - Good & Gather™ (Packaging May Vary)
Organic Honeycrisp Apples - 2lb Bag - Good & Gather™
Apple Pears - 36oz/3ct - Good & Gather™
Honeycrisp Apples - 3lb Bag - Good & Gather™
Hass Avocados - 4ct - Good & Gather™
Kid-Sized Bartlett Pears - 3lb Bag - Good & Gather™
Strawberries - 2lb
Jumbo Blueberries - 9.8oz
Organic Raspberries - 6oz
Organic Blueberries - 1pt
$3.59
$3.89
$5.39
$0.29
$3.89
$5.19
$2.49
$0.99
$5.49
$3.79
$0.99
$3.59
$0.79
$0.45
$2.99
$5.49
$6.00
$5.19
$3.89
$7.89
$6.49
$7.39
$4.89
$6.99


# creating a csv using all the urls(name, category, price)

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

#webdriver
chrome_path = 'C:\\Users\\nwang\\chromedriver.exe'
service = Service(chrome_path)
driver = webdriver.Chrome(service=service)

#define dict of categories using search urls
categories = {
    "Fresh Produce": "https://www.target.com/s?searchTerm=fresh+produce&tref=typeahead%7Cterm%7Cfresh+produce%7C%7C%7Chistory",
    "Meat & Seafood": "https://www.target.com/s?searchTerm=meat+and+seafood&tref=typeahead%7Cterm%7Cmeat+and+seafood%7C%7C%7Chistory",
    "Snacks": "https://www.target.com/s?searchTerm=snacks&tref=typeahead%7Cterm%7Csnacks%7C%7C%7Chistory",
    "Pantry": "https://www.target.com/s?searchTerm=pantry&tref=typeahead%7Cterm%7Cpantry%7C%7C%7Chistory",
    "Beverages": "https://www.target.com/s?searchTerm=beverages&tref=typeahead%7Cterm%7Cbeverages%7C%7C%7Chistory",
    "Breakfast & Cereal": "https://www.target.com/s?searchTerm=breakfast+and+cereals&tref=typeahead%7Cterm%7Cbreakfast+and+cereals%7C%7C%7Chistory",
    "Bakery": "https://www.target.com/s?searchTerm=bakery&tref=typeahead%7Cterm%7Cbakery%7C%7C%7Chistory",
    "Dairy & Eggs": "https://www.target.com/s?searchTerm=dairy+and+eggs&tref=typeahead%7Cterm%7Cdairy+and+eggs%7C%7C%7Chistory",
    "Frozen": "https://www.target.com/s?searchTerm=frozen&tref=typeahead%7Cterm%7Cfrozen%7C%7C%7Chistory"
}

#list to hold scraped data
data = []

for category, url in categories.items():
    driver.get(url)
    time.sleep(3)  #wait for page to load

    #scroll to reveal more products
    for _ in range(5):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)

    #find product price and name
    products = driver.find_elements(By.XPATH, '//a[@data-test="product-title"]')
    prices = driver.find_elements(By.XPATH, "//span[@data-test='current-price']//span")

    #appending each name, price, category to the list
    for product, price in zip(products, prices):
        name = product.get_attribute("aria-label")
        price_text = price.text
        data.append({
            "Category": category,
            "Name": name,
            "Price": price_text
        })

#creating a csv
df = pd.DataFrame(data)
df.to_csv('target_products.csv', index=False)

#close driver
driver.quit()

print("Data scraping complete. Products saved to 'target_products.csv'.")


Data scraping complete. Products saved to 'target_products.csv'.


# trying to add quantity


In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import re

#webdriver
chrome_path = 'C:\\Users\\nwang\\chromedriver.exe'
service = Service(chrome_path)
driver = webdriver.Chrome(service=service)

#define dict of categories using search urls
categories = {
    "Fresh Produce": "https://www.target.com/s?searchTerm=fresh+produce&tref=typeahead%7Cterm%7Cfresh+produce%7C%7C%7Chistory",
    "Meat & Seafood": "https://www.target.com/s?searchTerm=meat+and+seafood&tref=typeahead%7Cterm%7Cmeat+and+seafood%7C%7C%7Chistory",
    "Snacks": "https://www.target.com/s?searchTerm=snacks&tref=typeahead%7Cterm%7Csnacks%7C%7C%7Chistory",
    "Pantry": "https://www.target.com/s?searchTerm=pantry&tref=typeahead%7Cterm%7Cpantry%7C%7C%7Chistory",
    "Beverages": "https://www.target.com/s?searchTerm=beverages&tref=typeahead%7Cterm%7Cbeverages%7C%7C%7Chistory",
    "Breakfast & Cereal": "https://www.target.com/s?searchTerm=breakfast+and+cereals&tref=typeahead%7Cterm%7Cbreakfast+and+cereals%7C%7C%7Chistory",
    "Bakery": "https://www.target.com/s?searchTerm=bakery&tref=typeahead%7Cterm%7Cbakery%7C%7C%7Chistory",
    "Dairy & Eggs": "https://www.target.com/s?searchTerm=dairy+and+eggs&tref=typeahead%7Cterm%7Cdairy+and+eggs%7C%7C%7Chistory",
    "Frozen": "https://www.target.com/s?searchTerm=frozen&tref=typeahead%7Cterm%7Cfrozen%7C%7C%7Chistory"
}

# List to hold scraped product data
data = []

# Regex pattern to find quantities (e.g., "3 lbs", "1.13-2.75 lbs")
quantity_pattern = re.compile(r'(\d+\.?\d*\s*(?:lb|lbs|oz|ct|pk))', re.IGNORECASE)

for category, url in categories.items():
    driver.get(url)
    time.sleep(3)  # Wait for the page to load

    # Scroll to load products
    for _ in range(5):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)

    # Find product elements
    products = driver.find_elements(By.XPATH, '//a[@data-test="product-title"]')
    prices = driver.find_elements(By.XPATH, "//span[@data-test='current-price']//span")

    # Iterate through the products and prices
    for product, price in zip(products, prices):
        name = product.get_attribute("aria-label")
        price_text = price.text
        
        # Extract quantity from product name
        quantity_match = quantity_pattern.search(name)
        quantity_text = quantity_match.group(1) if quantity_match else "N/A"  # Default if quantity is not found

        # Append data to the list
        data.append({
            "Category": category,
            "Name": name,
            "Price": price_text,
            "Quantity": quantity_text
        })

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv('target_products_1.csv', index=False)

# Clean up and close the driver
driver.quit()

print("Data scraping complete. Products saved to 'target_products.csv'.")


Data scraping complete. Products saved to 'target_products.csv'.
